## Evaluation

### Intrinsic Evaluation

We measure the discrimination abilities of LLMs with four intrinsic metrics:

1. **Discrimination Accuracy (Acc)**: Given a pair of correct and wrong programs, we calculate the percentage where the correct program obtains a higher discrimination score than the wrong one.

2. **Classification Macro F1 (F1)**: We treat "correct" and "wrong" as two classes and compute the macro average of F1 scores on these two labels.

3. **Hit@1 (H@1)**: Given a batch of candidate programs, we calculate the percentage where the highest-scoring candidate is correct.

4. **Mean Reciprocal Rank (MRR)**: We compute the standard MRR score by the highest-ranking correct program in the batches.


Legacy script: `scripts\intrin_eval\intrin_eval_text2sql_ft.sh`

### Datasets:
1. [Spider](https://yale-lily.github.io/spider)

#### Spider Dataset Keys Explanation

The Spider dataset contains various keys that help in evaluating text-to-SQL models. Below is an explanation of each key:

- **`db_id`**: The unique identifier of the database for the given query. This indicates which database schema the question belongs to.

- **`schema`**: The schema of the database, which includes information about tables, columns, and their relationships. This helps models understand the database structure.

- **`question`**: The natural language question asked by the user.  
  *Example:*  
  *"What is the name of the youngest employee?"*

- **`sql`**: The ground truth SQL query corresponding to the question.  
  *Example:*  
  ```sql
  SELECT name FROM employees ORDER BY age ASC LIMIT 1;
  ```

- **`exec_res`**: The execution result of the ground truth SQL query. This contains the actual output of running the query on the database.

- **`top_n`**: A list of the **top-N SQL completions** (candidate queries) generated by the model. These are ranked based on the model’s confidence scores.

- **`top_n_exec_res`**: The execution results of the **top-N SQL completions**. These contain the actual database outputs of the model’s predicted queries.

- **`top_n_label`**: A list of binary labels (`0` or `1`) for each SQL candidate in `top_n`.  
  - `1` → The query is **correct** (produces the expected output).  
  - `0` → The query is **incorrect** (does not produce the expected output).

In [1]:
import os
from modules.llm_evaluator import LLMEvaluator, LLMLoraEvaluator, LLMReasoningEvaluator
from utils.functions import set_seed_all
from utils.functions import eval_intrinsic
import json
from datetime import datetime


In [2]:
evaluator_names =[
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0", # generation
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    "stabilityai/stable-code-3b", # generation
    "deepseek-ai/deepseek-coder-1.3b-base",
    "deepseek-ai/deepseek-coder-1.3b-instruct", # generation
    "codellama/CodeLlama-7b-Instruct-hf",
    "codellama/CodeLlama-13b-Instruct-hf"
]

# generate lora model names
lora_model_names = []
for m in evaluator_names:
   lora_model_names.append( m.split("/")[1]+"_spider")

In [ ]:
model_indx = 6 # choose the model to evaluate
evaluator_type = 'base' # 'base' / 'FT' / 'reason'
evaluator_setting = 'check' # 'base', 'check', 'exec', 'pro'
ft_type = 'NoExecResult' # 'NoExecResult' / 'withExecResult'
useSchema = False # Use schema as context for evaluator
evaluator_faliValue = -0.5
evaluator_max_new_tokens = 512

evaluator_name = evaluator_names[model_indx]
model_savename = lora_model_names[model_indx] + "_" + ft_type # + '_b256_e1'

print(f"evaluator_name: {evaluator_name}")
print(f"evaluator_lora: {model_savename}")
print(f"evaluator_type: {evaluator_type}")
print(f"evaluator setup: {evaluator_setting}")
print(f"FT training type: {ft_type}")
print(f"Use Schema: {useSchema}")
print(f"evaluator_max_new_tokens: {evaluator_max_new_tokens}")

current_directory = os.getcwd() #parameters
model_savedatapath = os.path.join(current_directory,f"checkpts/{model_savename}/model")
evaluator_peft_dir = model_savedatapath

seed = 42
curr_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#test_fname = "data/spider_intrin_eval.json"
test_fname = "data/bird_intrin_eval.json"
# test_fname = "data/spider_evaluator_train.json"
log_name = f"{model_savename}_{curr_timestamp}.json"
dataset_name = "spider"
db_path ="data/spider/database"
evaluation_config = f"configs/{evaluator_setting}.json"
print(f"evaluation_config = {json.load(open(evaluation_config))}")
"""
yes_token_indx: 
    the index of the token in the vocabulary that corresponds to the "Yes" text.
    CodeLlama-Instruct: "No" 1939 "Yes" 3869
    TinyLlama: "Yes" 3869
"""
yes_token_indx=None#3869

params_log={
   "evaluator_name": evaluator_name,
   "evaluator_lora": model_savename,
   "evaluator_type": evaluator_type,
   "evaluator_setting": evaluator_setting,
   "evaluation_config": json.load(open(evaluation_config)),
   "FT training type:": ft_type,
   "Use Schema in prompt": useSchema,
   "seed": seed,
   "test_fname": test_fname,
   "evaluator_faliValue": evaluator_faliValue,
   "evaluator_max_new_tokens": evaluator_max_new_tokens
}

print(params_log)

evaluator_name: codellama/CodeLlama-13b-Instruct-hf
evaluator_lora: CodeLlama-13b-Instruct-hf_spider_NoExecResult
evaluator_type: base
evaluator setup: check
FT training type: NoExecResult
Use Schema: False
evaluator_max_new_tokens: 512
evaluation_config = {'check_exec': True, 'use_exec_res': False}
{'evaluator_name': 'codellama/CodeLlama-13b-Instruct-hf', 'evaluator_lora': 'CodeLlama-13b-Instruct-hf_spider_NoExecResult', 'evaluator_type': 'base', 'evaluator_setting': 'check', 'evaluation_config': {'check_exec': True, 'use_exec_res': False}, 'FT training type:': 'NoExecResult', 'Use Schema in prompt': False, 'seed': 42, 'test_fname': 'data/spider_intrin_eval.json', 'evaluator_faliValue': -0.5, 'evaluator_max_new_tokens': 512}


In [4]:
# set seed
set_seed_all(seed)

In [5]:
if evaluator_type == 'base':
    evaluator = LLMEvaluator(evaluator_name, db_path, device="cuda",yes_token_indx=yes_token_indx)
    print(f"Loaded base model")
elif evaluator_type == 'FT':
    evaluator = LLMLoraEvaluator(evaluator_name, evaluator_peft_dir, db_path, device="cuda",yes_token_indx=yes_token_indx)
    print(f"Loaded LoRA FT model")
else:
    evaluator = LLMReasoningEvaluator(base_model_name=evaluator_name, db_path=db_path, device="cuda",\
                                       failvalue=evaluator_faliValue, max_new_tokens=evaluator_max_new_tokens)
    print(f"Loaded for reasoning")
# yindx=evaluator.get_yes_token()
# print(f"Yes token index: {yindx}")    

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded base model


https://stackoverflow.com/questions/78746073/how-to-solve-torch-was-not-compiled-with-flash-attention-warning

In [6]:
eval_intrinsic(evaluator, test_fname, evaluation_config, log_fname=log_name, useSchema=useSchema,params_log=params_log)

  0%|          | 0/400 [00:00<?, ?it/s]c:\Users\fahim\anaconda3\envs\r1\lib\site-packages\transformers\integrations\sdpa_attention.py:53: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 400/400 [25:39<00:00,  3.85s/it]


Pair Count: 409
PWS Acc: 0.7995              
SQL Count: 1221
Pos F1: 0.0000              
Neg F1: 0.7419              
Macro F1: 0.3709              
Hit @ 1: 0.6475              
MRR: 0.6777              

PWS Acc	Pos F1	Neg F1	Macro F1	Hit@1	MRR

0.7995              ,	0.0000              ,	0.7419              ,	0.3709              ,	0.6475              ,	0.6777              



In [7]:
# Clear all variables
%reset -f

# Clear memory for PyTorch (if using GPU)
import torch
torch.cuda.empty_cache()

# Garbage collection to free up memory
import gc
gc.collect()

0